# Analyzing Spectral Reflectance within Intertidal Zones

**Summary**
This tutorial demonstrates how to use Earth observation data from NASA's ECOSTRESS and EMIT missions alongside in situ spectral reflectance measurements to investigate environmental conditions and biodiversity in the intertidal zones of Southern California. Specifically, this notebook focuses on analyzing taxonomic diversity—such as algae types by comparing satellite data with groundtruthing reflectance spectra collected from tidepools. 

We aim to better understand how spectral signatures correspond to observable biological patterns by integrating:

- ECOSTRESS LST and emissivity layers (including LST, cloud mask, and quality control flags)

- EMIT full-band Level 2A reflectance data, and

- Normalized field-collected spectral data

Ultimately, the goal is to support coastal monitoring and climate adaptation efforts by incorporating various NASA missions

**Learning Objectives**

- What is spectral diversity and how does it relate to biodiversity?
- How to prepare and analyze reflectance data sets from on-ground and satellite observations. 
- Compare spectral diversity across sites and sensor types using summary statistics and visual plots.

_______________________________________________________________________________________________________________________________________________________

**Part 1: Data Setup and Dependencies**

Create a working environment with required packages. Be sure to active your environment and choose the right kernel each time you work on your project.

In [1]:
#Run in Terminal: mamba create -n (environmentname) -c conda-forge --yes python=3.12 gdal fiona hvplot geoviews rioxarray rasterio jupyter geopandas earthaccess jupyter_bokeh h5py h5netcdf spectral scikit-image jupyterlab seaborn dask ray-default pystac-client odc-stac pyresample libgdal-hdf4 harmony-py

# Import required libraries
import os
import json
import folium
import earthaccess
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import cartopy.crs as ccrs

from branca.element import Figure
from shapely.geometry import MultiPolygon, Polygon, box
from shapely.geometry.polygon import orient
from datetime import timedelta

**Part 2: NASA Earthdata Authentication**

To access data from NASA Earthdata, you must first authenticate using the earthaccess library. Follow the interactive login prompt that appears to sign in with your NASA Earthdata credentials (email and password associated with your Earthdata account). This step ensures you have authorized access to download and search Earth observation datasets.

If you haven’t registered yet, create a free account here: https://urs.earthdata.nasa.gov/users/new

In [2]:
# Authenticate earthaccess
auth = earthaccess.Auth()
auth.login(strategy="interactive", persist=True)
print('Authenticated:', auth.authenticated)

Authenticated: True


**Part 3: Loading In Situ Data**

In order to analyze field sites, in situ data must be loaded in. Replace the file names with your own geojson In Situ file. If only remote sensing practices will be used, commment out this code.

In [ ]:
#Define Region(s) of Interest for ECOSTRESS and EMIT Requests. 
%matplotlib inline
LCDMPolygon = gpd.read_file("/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/LCDMPolygon.geojson")
SDPolygon = gpd.read_file("/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/SDPolygon.geojson")
PVPolygon = gpd.read_file("/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/PVPolygon.geojson")

print(LCDMPolygon.head())
print(SDPolygon.head())
print(PVPolygon.head())

LCDMPolygon.plot() #not necessary to plot. 
SDPolygon.plot()
PVPolygon.plot()

**Part 3a: Visualizing Sample Locations**

It is helpful to validate whether your data was loaded in properly. Visualizing your AOI will also assist in analysis.

In [ ]:
#Visualize Sample Locations Within One Grid

# Load GeoJSON files
LCDMQuadrat = gpd.read_file("/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/LCDMQuadrat.geojson")
SDQuadrat = gpd.read_file("/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/SDQuadrat.geojson")
PVQuadrat = gpd.read_file("/Users/kylamonique/Desktop/JPLFiles/SpectralEvolution/GIS/Waypoints/PVQuadrat.geojson")

# Print sample rows
print(LCDMQuadrat.head())
print(SDQuadrat.head())
print(PVQuadrat.head())

# Set up the plot with Cartopy for mapping
fig, ax = plt.subplots(figsize=(12, 8), subplot_kw={'projection': ccrs.PlateCarree()})

# Add high-resolution coastlines and gridlines
ax.coastlines(resolution='10m')
gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
gl.top_labels = gl.right_labels = False

# Plot the GeoDataFrames
LCDMQuadrat.plot(ax=ax, marker='o', color='red', label='LCDM', transform=ccrs.PlateCarree())
SDQuadrat.plot(ax=ax, marker='^', color='green', label='SD', transform=ccrs.PlateCarree())
PVQuadrat.plot(ax=ax, marker='s', color='blue', label='PV', transform=ccrs.PlateCarree())

plt.legend()
plt.title("Site Sampling Locations")
plt.show()


# Visualize quadrat locations per site.
# def plot_quadrats(gdf, title, color, marker):
#     fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
#     ax.coastlines(resolution='10m')
    
#     # Add gridlines
#     gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
#     gl.top_labels = gl.right_labels = False

#     # Add high-resolution coastlines and gridlines
#     ax.coastlines(resolution='10m')
#     gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
#     gl.top_labels = gl.right_labels = False
    
#     # Plot site
#     gdf.plot(ax=ax, color=color, marker=marker, alpha=0.8, transform=ccrs.PlateCarree(), label=title)
#     plt.title(title)
#     plt.legend()
#     plt.show()

def plot_quadrats(gdf, title, color, marker):
    fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
    
    # Plot coastlines
    ax.coastlines(resolution='10m')
    
    # Gridlines
    gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5)
    gl.top_labels = gl.right_labels = False

    # Plot quadrats
    gdf.plot(ax=ax, color=color, marker=marker, alpha=0.8, transform=ccrs.PlateCarree(), label=title)

    # Auto-zoom to the bounds of the GeoDataFrame (pad for clarity)
    minx, miny, maxx, maxy = gdf.total_bounds
    padding = 0.0001 # degree buffer
    ax.set_extent([minx - padding, maxx + padding, miny - padding, maxy + padding], crs=ccrs.PlateCarree())

    # Title and legend
    plt.title(title)
    plt.legend()
    plt.show()


# Plot each one separately
plot_quadrats(LCDMQuadrat, "LCDM Quadrat Sites", "red", "o")
plot_quadrats(SDQuadrat,   "SD Quadrat Sites",   "green", "^")
plot_quadrats(PVQuadrat,   "PV Quadrat Sites",   "blue", "s")


**Part 4: Accessing and Utilizing ECOSTRESS and EMIT Data**